<a href="https://colab.research.google.com/github/hws2002/MachineLearning_PytorchNScikitLearn/blob/master/Chapter6/Chapter6_1_pipeline_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6.1 파이프라인을 사용한 효율적인 워크플로

사이킷런의 pipeline 클래스는 여러 개의 변환단계를  포함한 모델을 학습하고 새로운 데이터에 대한 예측을 만들 수 있음.

## 6.1.1 위스콘신 유방암 데이터셋
이 장에서는 위스콘신 유방암 데이터셋을 사용할 것임.  
여기에는 악성과 양성인 종양 세포 샘플 569개가 포함되어 있음.  

데이터셋 첫 두 열에는 고유 ID번호와 진단결과( M = 악성, B = 양성) 이 들어 있고, 세 번째에서 32번째까지 열에는 세포 핵의 디지털 이미지에서 계산된 30개의 실수 값 특성이 담겨 있음.  

이 특성을 사용하여 종양이 악성인지 양성인지 예측하는 모델을 만들어 볼 것임.  

데이터셋에 대한 자세한 내용은 https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic 를 참고하자

In [7]:
import pandas as pd
df = pd.read_csv(
    'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data',
    header = None
)

In [8]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [45]:
df.columns = [
    'ID',
    'Diagnosis',
    'radius1', 'texture1', 'perimeter1', 'area1', 'smoothness1', 'compactness1', 'concavity1', 'concave_points1', 'symmetry1', 'fractal_dimension1',
    'radius2', 'texture2', 'perimeter2', 'area2', 'smoothness2', 'compactness2', 'concavity2', 'concave_points2', 'symmetry2', 'fractal_dimension2',
    'radius3', 'texture3', 'perimeter3', 'area3', 'smoothness3', 'compactness3', 'concavity3', 'concave_points3', 'symmetry3', 'fractal_dimension3'
]
df.head()

,ID,Diagnosis,radius1,texture1,perimeter1,area1,smoothness1,compactness1,concavity1,concave_points1,...,radius3,texture3,perimeter3,area3,smoothness3,compactness3,concavity3,concave_points3,symmetry3,fractal_dimension3
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [46]:
X, y = df.iloc[:,2:].values , df.iloc[:,1].values

In [47]:
# labelEncoder로 클래스 레이블을 (M,B)에서 정수로 변환
print(y[:10])
## 수동
import numpy as np
class_mapping = {
    label : idx for idx, label in enumerate(np.unique(y))
}
print(class_mapping)
y_encoded = np.array([class_mapping[label] for label in y])
print(y_encoded[:10])
## sklearn
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
print(y[:10])
print(le.classes_)
# M : 1, B : 0 으로 변환되었음을 알 수 있다.

['M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M']
{'B': 0, 'M': 1}
[1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1]
['B' 'M']


In [48]:
# 훈련 데이터셋과 테스트 데이터셋 80/20 을 나눈다
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, stratify = y, random_state = 1)

## 6.1.2 파이프라인으로 변환기와 추정기 연결
standardScaler, PCA, LogisticRegression객체를 하나의 파이프라인으로 연결해보자.

In [50]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

pipe_lr = make_pipeline( StandardScaler(),
                        PCA(n_components=2),
                        LogisticRegression(random_state = 1))
pipe_lr.fit(X_train, y_train)
y_pred = pipe_lr.predict(X_test)
test_acc = pipe_lr.score(X_test, y_test)
print(f'테스트 정확도 : {test_acc:.3f}')

테스트 정확도 : 0.956


In [52]:
from sklearn import set_config
set_config(display = 'diagram')
pipe_lr

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=2)),
                ('logisticregression', LogisticRegression(random_state=1))])